In [187]:
import cv2
import numpy as np
import random
import torch
from skimage.util import random_noise
import os

In [99]:
def preprocess_base(original):
    img = cv2.imread(original)
    h,w,d = img.shape
    nh = int(h*600/w)
    nw = int(w*400/h)
    if (nh>400):
        img = cv2.resize(img, (600,nh))
    else:
        img = cv2.resize(img, (nw,400))
    h,w,d = img.shape
    img_cpd = img[h//2-200:h//2+200,w//2-300:w//2+300]
    return img_cpd
def preprocess_icons(original):
    img = cv2.imread(original,cv2.IMREAD_UNCHANGED)
    h,w,d = img.shape
    img = cv2.resize(img, (12,12))
    return img

In [100]:
bases = []
for i in range(3):
    basex = preprocess_base(f"base{i}.png")
    bases.append(basex)
    cv2.imwrite(os.path.join(os.getcwd(),"bases",f"base{i}.png"),basex)
icons = []
for i in range(1):
    iconx = preprocess_icons(f"icon{i}.png")
    icons.append(iconx)
    cv2.imwrite(os.path.join(os.getcwd(),"icons",f"icon{i}.png"),iconx)

In [288]:
def recolor(img):
    import random
#     img = cv2.imread(original)
    b = random.random()*0.6+0.4
    g = random.random()*(1-b)
    r = 1-g-b
    h,w,d = img.shape
    for i in range(h):
        for j in range(w):
            s = sum(list(img[i][j]))
            img[i][j] = np.array(recolor_pix([b,g,r],s))
    # cv2.imwrite(new,img)
    return img

def recolor_pix(coloring, original):
    b,g,r = [(coloring[i]*original) for i in range(3)]
    g = g+max(0,b-255)
    r = r+max(0,g-255)
    b = b+max(0,r-255)
    return [min(255,b),min(255,g),min(255,r)]
        

In [66]:
recolor("base2.png","test?.png")

In [182]:
def generate_loc(img,size):
    brightness = -70
    contrast = 2
    img2 = cv2.addWeighted(img, contrast, np.zeros(img.shape, img.dtype), contrast, brightness)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    locs = []
    while size>0:
        x,y = random.randint(50,550), random.randint(50,350)
        if img2[y][x] == 255:
            continue
        else :
            size-=1
            locs.append([x,y])
    return locs

def generate_edges_Bidirectional(num_edges,num_nodes):
    adj = [set() for i in range(num_nodes)]
    nodes = set(range(num_nodes))
    count = 0
    while num_edges > count:
        source = random.randint(0,num_nodes-1)
        targets = list(nodes - adj[source])
        if len(targets)>0:
            target = random.sample(targets,1).pop()
            adj[source].add(target)
            adj[target].add(source)
            count += 1
    return adj
def plot_nodes_and_edges(img,num_edges,num_nodes,node_clr,edge_clr):
    nodes = generate_loc(img,num_nodes)
    adj = generate_edges_Bidirectional(num_edges,num_nodes)
    for i in range(num_nodes):
        adjacents = adj[i]
        while adjacents:
            j = adjacents.pop()
            if (i<j):
                img = cv2.line(img,nodes[i],nodes[j],edge_clr,2)
        img = cv2.circle(img,nodes[i],5,node_clr,-1)
    return img    
    

img = recolor("base.png")
img_new = plot_nodes_and_edges(img,20,10,(255,0,0),(0,0,255))
cv2.imwrite("blehhh.png",img_new)

True

In [289]:
colors = [(37,37,37),(204,79,73),(255,255,255),(50,50,229)]
path = os.path.join(os.getcwd(),"syndata")
if not os.path.exists(path):
    os.makedirs(path)
for b in range(3):
    base = cv2.imread(os.path.join(os.getcwd(),"bases",f"base{b}.png"))
    for i in tqdm(range(4)):
        base_rclr = recolor(base)
        for j in range(50):
            num_nodes = random.randint(3,15)
            num_edges = random.randint(5,num_nodes*2)
            node_clr,edge_clr = random.sample(colors,2)
            img_new = base_rclr.copy()
            img_new = plot_nodes_and_edges(img_new,num_edges,num_nodes,node_clr,edge_clr)
            cv2.imwrite(os.path.join(path,f"syn{b}-{i}-{j}.png"),img_new)

100%|█████████████████████████████████████████████| 4/4 [00:06<00:00,  1.72s/it]


In [325]:
import cv2
import numpy as np
import os
from tqdm import tqdm
import random
from skimage.util import random_noise

def noise_Guassian(img):
    img = random_noise(img, mode='gaussian', mean=0, var=0.01, clip=True)
    img = (255*img).astype(np.uint8)
    return img
def noise_SP(img):
    img = random_noise(img, mode='s&p', salt_vs_pepper=0.5, clip=True)
    img = (255*img).astype(np.uint8)
    return img
def guass_blur(img):
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img
def resize_blur(img):
    h,w,d = img.shape
    img = cv2.resize(img,(int(w*1.2),int(h*1.2)),interpolation = cv2.INTER_LINEAR)
    img = cv2.resize(img,(w,h),interpolation = cv2.INTER_LINEAR)
    return img
def random_crop(img):
    h,w,d = img.shape
    sx,sy = random.randint(0,h//4), random.randint(0,w//4)
    ex,ey = sx + random.randint(h//2,(h//4)*3), sy + random.randint(w//2,(w//4)*3)
    img = img[sx:ex,sy:ey]
    return img
def flip(img):
    h,w,d = img.shape
    for i in range(h):
        for j in range(w//2):
            img[i][j], img[i][w-j-1] = img[i][w-j-1].copy(), img[i][j].copy()
    return img

In [368]:
def noise_augment(img,crop=True,canflip=True,blur=True):
    if (random.random() < 0.5):
        img = noise_Guassian(img)
    else:
        img = noise_SP(img)
    if crop:
        img = random_crop(img)
    if (random.random()<0.5 and canflip):
        img = flip(img)
    if (random.random()<0.5 and blur):
        img = guass_blur(img)
    return img

if __name__=="__main__":
    path = input("Image directory path:")
    newpath = os.path.join(path,"noise_augmented_imgs")
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    supported_formats = ["jpg",'png','webp']
    dir_list = os.listdir(path)
    for i in tqdm(range(len(dir_list))):
        f = dir_list[i]
        if f.split(".")[-1] in supported_formats:
            img = cv2.imread(os.path.join(path,f))
            img = resize_blur(img)
            img = guass_blur(img)
            img = noise_augment(img,blur = False,crop=False,canflip=False)
            cv2.imwrite(os.path.join(newpath,f),img)
            

Image directory path:/Users/sherry/Downloads/cv/syndata_curved_w_data


100%|█████████████████████████████████████████| 604/604 [00:16<00:00, 36.05it/s]


In [340]:
def hanging_line(point1, point2):
    import numpy as np

    a = (point2[1] - point1[1])/(np.cosh(point2[0]) - np.cosh(point1[0]))
    b = point1[1] - a*np.cosh(point1[0])
    x = np.linspace(point1[0], point2[0], 500)
    y = a*np.cosh(x) + b

    return (x,y)

def plot_hanging_line(img,a,b,clr):
    pa = [i/100 for i in a]
    pb = [j/100 for j in b]
    h,w,d = img.shape
    x,y = hanging_line([w/100 - pa[0], h/100 - pa[1]],[w/100 - pb[0],h/100 - pb[1]])
    for i in range(len(x)):
        img[h-int(y[i]*100)][w-int(x[i]*100)] = clr
    return img
    
def plot_reverse_hanging_line(img,a,b,clr):
    pa = [i/100 for i in a]
    pb = [j/100 for j in b]
    x,y = hanging_line(pa,pb)
    for i in range(len(x)):
        img[int(y[i]*100)][int(x[i]*100)] = clr
    return img
    
def plot_nodes_and_edges_curved(img,num_edges,num_nodes,node_clr,edge_clr,randomness=0.5):
    nodes = generate_loc(img,num_nodes)
    node_size = random.randint(2,6)
    adj = generate_edges_Bidirectional(num_edges,num_nodes)
    for i in range(num_nodes):
        adjacents = adj[i]
        while adjacents:
            j = adjacents.pop()
            if (i<j):
                if (nodes[i][0] == nodes[j][0] or nodes[i][1]==nodes[j][1]):
                    img = cv2.line(img,nodes[i],nodes[j],edge_clr,1)
                    continue
                reverse = random.random()
                if reverse<randomness:
                    img = plot_reverse_hanging_line(img,nodes[i],nodes[j],edge_clr)
                else:
                    img = plot_hanging_line(img,nodes[i],nodes[j],edge_clr)
        img = cv2.circle(img,nodes[i],node_size,node_clr,-1)
    return img 

In [341]:
colors = [(37,37,37),(204,79,73),(255,255,255),(50,50,229)]
path = os.path.join(os.getcwd(),"syndata_curved")
if not os.path.exists(path):
    os.makedirs(path)
for b in range(3):
    base = cv2.imread(os.path.join(os.getcwd(),"bases",f"base{b}.png"))
    for i in tqdm(range(4)):
        base_rclr = recolor(base)
        for j in range(50):
            num_nodes = random.randint(3,15)
            num_edges = random.randint(5,num_nodes*2)
            node_clr,edge_clr = random.sample(colors,2)
            img_new = base_rclr.copy()
            img_new = plot_nodes_and_edges_curved(img_new,num_edges,num_nodes,node_clr,edge_clr)
            cv2.imwrite(os.path.join(path,f"syn{b}-{i}-{j}.png"),img_new)

100%|█████████████████████████████████████████████| 4/4 [00:05<00:00,  1.30s/it]


In [369]:
def plot_nodes_and_edges_curved_data(img,num_edges,num_nodes,node_clr,edge_clr,randomness=0.5):
    nodes = generate_loc(img,num_nodes)
    node_size = random.randint(2,6)
    adj = generate_edges_Bidirectional(num_edges,num_nodes)
    edgs = []
    for i in range(num_nodes):
        adjacents = adj[i]
        while adjacents:
            j = adjacents.pop()
            if (i<j):
                edgs.append([i,j])
                if (nodes[i][0] == nodes[j][0] or nodes[i][1]==nodes[j][1]):
                    img = cv2.line(img,nodes[i],nodes[j],edge_clr,1)
                    continue
                reverse = random.random()
                if reverse<randomness:
                    img = plot_reverse_hanging_line(img,nodes[i],nodes[j],edge_clr)
                else:
                    img = plot_hanging_line(img,nodes[i],nodes[j],edge_clr)
        img = cv2.circle(img,nodes[i],node_size,node_clr,-1)
    return img,nodes,edgs


In [367]:
import csv
import pandas as pd
colors = [(37,37,37),(204,79,73),(179,0,98),(50,50,229),(40,6,41),(118,99,251)]
path = os.path.join(os.getcwd(),"syndata_curved_w_data")
if not os.path.exists(path):
    os.makedirs(path)
fields = ["img_name","nodes","edges"]
rows = []
for b in range(3):
    base = cv2.imread(os.path.join(os.getcwd(),"bases",f"base{b}.png"))
    for i in tqdm(range(4)):
        base_rclr = recolor(base)
        for j in range(50):
            num_nodes = random.randint(3,15)
            num_edges = random.randint(5,num_nodes*2)
            node_clr,edge_clr = random.sample(colors,2)
            img_new = base_rclr.copy()
            img_new, nodes, adj = plot_nodes_and_edges_curved_data(img_new,num_edges,num_nodes,node_clr,edge_clr)
            rows.append([f"syn{b}-{i}-{j}.png",nodes,adj])
            cv2.imwrite(os.path.join(path,f"syn{b}-{i}-{j}.png"),img_new)

filename = os.path.join(path,"data.csv")
with open(filename, 'w') as csvfile:   
    csvwriter = csv.writer(csvfile)   
    csvwriter.writerow(fields) 
    csvwriter.writerows(rows) 
    
df = pd.DataFrame(rows, columns = ["Name","Nodes","Edges"])
df.to_pickle(os.path.join(path,"data.pkl"))

100%|█████████████████████████████████████████████| 4/4 [00:07<00:00,  1.96s/it]
